In [1]:
import warnings
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
warnings.filterwarnings('ignore')
House_Sub=pd.read_csv('Ames Real Estate Data.csv')
House_Main=pd.read_csv('Ames_HousePrice.csv',index_col=0)

In [3]:
House_Main.describe(include='all')

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
count,2.580000e+03,2580.000000,2580.000000,2580.000000,2580,2118.000000,2580.000000,2580,168,2580,...,2580.000000,2580.000000,9,525,97,2580.000000,2580.000000,2580.000000,2580,2580
unique,NaN,NaN,NaN,NaN,7,NaN,NaN,2,2,4,...,NaN,NaN,4,4,4,NaN,NaN,NaN,10,6
top,NaN,NaN,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,...,NaN,NaN,TA,MnPrv,Shed,NaN,NaN,NaN,WD,Normal
freq,NaN,NaN,NaN,NaN,2008,NaN,NaN,2570,105,1641,...,NaN,NaN,3,301,88,NaN,NaN,NaN,2392,2414
mean,7.148300e+08,1486.039922,178059.623256,57.691860,NaN,68.516053,10120.153488,NaN,NaN,NaN,...,16.200388,1.662016,NaN,NaN,NaN,48.731395,6.150775,2007.838372,NaN,NaN
std,1.886626e+08,488.650181,75031.089374,42.802105,NaN,22.835831,8126.937892,NaN,NaN,NaN,...,56.824783,30.339396,NaN,NaN,NaN,498.725058,2.670647,1.312333,NaN,NaN
min,5.263011e+08,334.000000,12789.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN
25%,5.313630e+08,1112.000000,129975.000000,20.000000,NaN,57.000000,7406.750000,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,4.000000,2007.000000,NaN,NaN
50%,5.354546e+08,1436.000000,159900.000000,50.000000,NaN,68.000000,9391.000000,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN
75%,9.071811e+08,1733.000000,209625.000000,70.000000,NaN,80.000000,11494.000000,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN


# Find Columns whose dtypes are numeric and initial cor analysis of those columns to target: SalePrice

In [4]:
#The target dependent varibale for this case is SalePrice column in House_Main table. In order to do some initial analysis, 
#we find columns that are of numeric type first.
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
House_Main_Num=House_Main.select_dtypes(include=numerics)

In [5]:
#PID is not useful as it's working as an ID index, we drop that column
House_Main_Num=House_Main_Num.drop(['PID'],axis=1)

In [6]:
#Do the correlation analysis of the numeric variables

#set the display of output table/dataframe to max columns
pd.set_option('display.max_columns',None)

#Find the correlation of all numeric variables
cormax=round(House_Main_Num.corr(),2)

In [7]:
cormax.head()

,GrLivArea,SalePrice,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
GrLivArea,1.00,0.72,0.07,0.35,0.26,0.56,-0.11,0.23,0.31,0.39,0.16,-0.02,0.25,0.41,0.54,0.67,0.09,0.04,-0.05,0.64,0.45,0.54,0.13,0.81,0.46,0.27,0.50,0.48,0.25,0.32,0.00,0.01,0.10,0.06,-0.00,0.04,-0.01
SalePrice,0.72,1.00,-0.08,0.36,0.27,0.79,-0.10,0.54,0.51,0.50,0.46,0.02,0.17,0.65,0.64,0.26,-0.03,0.29,-0.03,0.53,0.29,0.15,-0.11,0.49,0.49,0.52,0.64,0.63,0.33,0.31,-0.12,0.03,0.12,0.03,-0.02,0.01,-0.00
MSSubClass,0.07,-0.08,1.00,-0.44,-0.20,0.06,-0.08,0.05,0.06,0.02,-0.07,-0.07,-0.11,-0.22,-0.26,0.31,0.01,-0.00,-0.01,0.14,0.18,-0.02,0.25,0.03,-0.05,0.11,-0.03,-0.09,-0.02,-0.01,-0.03,-0.04,-0.06,-0.01,-0.03,0.01,-0.03
LotFrontage,0.35,0.36,-0.44,1.00,0.46,0.18,-0.05,0.10,0.06,0.20,0.18,0.06,0.10,0.31,0.44,0.01,-0.01,0.11,-0.03,0.17,0.03,0.25,0.02,0.34,0.25,0.05,0.30,0.34,0.12,0.14,0.00,0.03,0.08,0.09,0.02,0.00,-0.00
LotArea,0.26,0.27,-0.20,0.46,1.00,0.08,-0.03,0.01,0.01,0.10,0.16,0.08,0.02,0.22,0.31,0.02,-0.00,0.12,0.03,0.12,0.03,0.14,-0.01,0.20,0.25,-0.02,0.18,0.20,0.16,0.08,0.02,0.02,0.05,0.05,0.04,0.01,-0.02


In [8]:
#From the above table we find the correlation of SalePrice and other independent numeric varibales
cormax=cormax['SalePrice']
cormax

GrLivArea        0.72
SalePrice        1.00
MSSubClass      -0.08
LotFrontage      0.36
LotArea          0.27
OverallQual      0.79
OverallCond     -0.10
YearBuilt        0.54
YearRemodAdd     0.51
MasVnrArea       0.50
BsmtFinSF1       0.46
BsmtFinSF2       0.02
BsmtUnfSF        0.17
TotalBsmtSF      0.65
1stFlrSF         0.64
2ndFlrSF         0.26
LowQualFinSF    -0.03
BsmtFullBath     0.29
BsmtHalfBath    -0.03
FullBath         0.53
HalfBath         0.29
BedroomAbvGr     0.15
KitchenAbvGr    -0.11
TotRmsAbvGrd     0.49
Fireplaces       0.49
GarageYrBlt      0.52
GarageCars       0.64
GarageArea       0.63
WoodDeckSF       0.33
OpenPorchSF      0.31
EnclosedPorch   -0.12
3SsnPorch        0.03
ScreenPorch      0.12
PoolArea         0.03
MiscVal         -0.02
MoSold           0.01
YrSold          -0.00
Name: SalePrice, dtype: float64

In [9]:
#We sort the correlation from highest to lowest
np.abs(cormax).sort_values(ascending=False)

SalePrice        1.00
OverallQual      0.79
GrLivArea        0.72
TotalBsmtSF      0.65
1stFlrSF         0.64
GarageCars       0.64
GarageArea       0.63
YearBuilt        0.54
FullBath         0.53
GarageYrBlt      0.52
YearRemodAdd     0.51
MasVnrArea       0.50
Fireplaces       0.49
TotRmsAbvGrd     0.49
BsmtFinSF1       0.46
LotFrontage      0.36
WoodDeckSF       0.33
OpenPorchSF      0.31
BsmtFullBath     0.29
HalfBath         0.29
LotArea          0.27
2ndFlrSF         0.26
BsmtUnfSF        0.17
BedroomAbvGr     0.15
ScreenPorch      0.12
EnclosedPorch    0.12
KitchenAbvGr     0.11
OverallCond      0.10
MSSubClass       0.08
3SsnPorch        0.03
PoolArea         0.03
BsmtHalfBath     0.03
LowQualFinSF     0.03
BsmtFinSF2       0.02
MiscVal          0.02
MoSold           0.01
YrSold           0.00
Name: SalePrice, dtype: float64

# Find columns whose dtypes are non-numeric and label-encode or dummify variables in those columns

In [10]:
#Get all the columns whose dtypes are not numeric, i.e. object
House_Main_Nonnum=House_Main.select_dtypes(exclude=numerics)
House_Main_Nonnum.dtypes

MSZoning         object
Street           object
Alley            object
LotShape         object
LandContour      object
Utilities        object
LotConfig        object
LandSlope        object
Neighborhood     object
Condition1       object
Condition2       object
BldgType         object
HouseStyle       object
RoofStyle        object
RoofMatl         object
Exterior1st      object
Exterior2nd      object
MasVnrType       object
ExterQual        object
ExterCond        object
Foundation       object
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Heating          object
HeatingQC        object
CentralAir       object
Electrical       object
KitchenQual      object
Functional       object
FireplaceQu      object
GarageType       object
GarageFinish     object
GarageQual       object
GarageCond       object
PavedDrive       object
PoolQC           object
Fence            object
MiscFeature      object
SaleType        

In [11]:
House_Main_Nonnum.shape[1] #We know there are 43 variables whose dtypes are 'object'

43

In [12]:
House_Main_Nonnum.isnull().sum(axis=0) #From the list below, we know some object variables have null-type values.

MSZoning            0
Street              0
Alley            2412
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType         14
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           69
BsmtCond           69
BsmtExposure       71
BsmtFinType1       69
BsmtFinType2       70
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         0
Functional          0
FireplaceQu      1241
GarageType        127
GarageFinish      129
GarageQual        129
GarageCond        129
PavedDrive          0
PoolQC           2571
Fence            2055
MiscFeature      2483
SaleType            0
SaleCondition       0
dtype: int64

In [13]:
#Subset the House_Main_Nonnum columns that do not have nan values
House_Main_Nonnum_Init=House_Main_Nonnum[House_Main_Nonnum.columns[~House_Main_Nonnum.isnull().any()]]

House_Main_Nonnum_Init.isnull().sum(axis=0)

MSZoning         0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
ExterQual        0
ExterCond        0
Foundation       0
Heating          0
HeatingQC        0
CentralAir       0
KitchenQual      0
Functional       0
PavedDrive       0
SaleType         0
SaleCondition    0
dtype: int64

In [17]:
#We do a value counts here to make sure words are not used to represent numbers (like 'One', 'Two', ...) If there are we 
#could just replace with integers. (The result shows none)
House_Main_Nonnum_Init.value_counts

<bound method DataFrame.value_counts of     MSZoning Street LotShape LandContour Utilities LotConfig LandSlope  \
1         RL   Pave      Reg         Lvl    AllPub    Corner       Gtl   
2         RL   Pave      Reg         Lvl    AllPub    Inside       Gtl   
3    C (all)   Pave      Reg         Lvl    AllPub    Inside       Gtl   
4         RL   Pave      Reg         Lvl    AllPub    Corner       Gtl   
5         RL   Pave      Reg         Lvl    AllPub    Inside       Gtl   
..       ...    ...      ...         ...       ...       ...       ...   
759       RL   Pave      Reg         Lvl    AllPub    Inside       Gtl   
760       RL   Pave      IR1         Lvl    AllPub   CulDSac       Gtl   
761       RH   Pave      Reg         HLS    AllPub    Inside       Gtl   
762       RL   Pave      Reg         Lvl    AllPub    Inside       Gtl   
763       RL   Pave      IR1         Lvl    AllPub    Inside       Gtl   

    Neighborhood Condition1 Condition2 BldgType HouseStyle RoofStyle Ro

In [23]:
#Making sure there's no empty cells in the columns of House_Main_Nonnum_Init
for col in House_Main_Nonnum_Init:
    print(col,(House_Main_Nonnum_Init[col].values=='').sum())

MSZoning 0
Street 0
LotShape 0
LandContour 0
Utilities 0
LotConfig 0
LandSlope 0
Neighborhood 0
Condition1 0
Condition2 0
BldgType 0
HouseStyle 0
RoofStyle 0
RoofMatl 0
Exterior1st 0
Exterior2nd 0
ExterQual 0
ExterCond 0
Foundation 0
Heating 0
HeatingQC 0
CentralAir 0
KitchenQual 0
Functional 0
PavedDrive 0
SaleType 0
SaleCondition 0


In [18]:
for col in House_Main_Nonnum_Init:
    print(col,House_Main_Nonnum_Init[col].unique())

MSZoning ['RL' 'C (all)' 'RM' 'FV' 'RH' 'I (all)' 'A (agr)']
Street ['Pave' 'Grvl']
LotShape ['Reg' 'IR1' 'IR3' 'IR2']
LandContour ['Lvl' 'Bnk' 'HLS' 'Low']
Utilities ['AllPub' 'NoSewr']
LotConfig ['Corner' 'Inside' 'FR2' 'CulDSac' 'FR3']
LandSlope ['Gtl' 'Mod' 'Sev']
Neighborhood ['SWISU' 'Edwards' 'IDOTRR' 'OldTown' 'NWAmes' 'Blmngtn' 'Mitchel'
 'NridgHt' 'NAmes' 'CollgCr' 'SawyerW' 'MeadowV' 'BrDale' 'Gilbert'
 'Timber' 'Somerst' 'BrkSide' 'NoRidge' 'Sawyer' 'Crawfor' 'Greens'
 'ClearCr' 'GrnHill' 'StoneBr' 'Veenker' 'NPkVill' 'Blueste' 'Landmrk']
Condition1 ['Norm' 'RRAn' 'PosN' 'Artery' 'RRAe' 'Feedr' 'PosA' 'RRNn' 'RRNe']
Condition2 ['Norm' 'PosN' 'Feedr' 'Artery' 'PosA' 'RRNn' 'RRAn' 'RRAe']
BldgType ['1Fam' 'TwnhsE' '2fmCon' 'Twnhs' 'Duplex']
HouseStyle ['1Story' '2Story' 'SFoyer' 'SLvl' '1.5Fin' '2.5Unf' '1.5Unf' '2.5Fin']
RoofStyle ['Gable' 'Hip' 'Mansard' 'Gambrel' 'Flat' 'Shed']
RoofMatl ['CompShg' 'Tar&Grv' 'WdShake' 'WdShngl' 'Metal' 'Roll' 'Membran']
Exterior1st ['Wd Sdn

In [24]:
#We first take care of variables that are suitable for binary encoding:
Binary=['Street','Utilities','CentralAir']
#Then we look at variables that are suitable for Dummy encoding are the rest
#Of the names in Binary, CentralAir is Y/N type of binary encoding.

In [25]:
# Convert CentralAir column to '1' and '0' binary type.
House_Main_Nonnum_Init['CentralAir']=np.where(House_Main_Nonnum_Init['CentralAir'].str.contains('Y'),1,0)

In [30]:
# Label Encoding Street and Utilities columns. For Street, 1 means Pave; For Utilities, 0 means AllPub. 
House_Main_Nonnum_Init['Street']=House_Main_Nonnum_Init['Street'].astype('category')
House_Main_Nonnum_Init['Utilities']=House_Main_Nonnum_Init['Utilities'].astype('category')
House_Main_Nonnum_Init['Street']=House_Main_Nonnum_Init['Street'].cat.codes
House_Main_Nonnum_Init['Utilities']=House_Main_Nonnum_Init['Utilities'].cat.codes

In [ ]:
#Before we dummy-encoding the remaining object type variables, we noticed there are some that contain so many values.


In [50]:
pd.get_dummies(House_Main_Nonnum_Init.loc[:,~House_Main_Nonnum_Init.columns.isin(Binary)],drop_first=True)

,MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_Po,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_VWD,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
5,0,0,0,0,1,0,0,0,1,0,0,1,0,0